In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [24]:
# Load the trained model, scaler pickle, OHE
model = load_model('model.h5')

# Load the encoder and scaler
with open('OHE.pkl', 'rb') as file:
    OHE = pickle.load(file)
    
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [5]:
input_data = {
    'CreditScore':500,
    'Geography':'France',
    'Gender':'male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrcard':1,
    'IsActiveMember':1,
    'EstimatedSalary': 50000    
}

In [6]:
geo_encoded = OHE.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=OHE.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrcard,IsActiveMember,EstimatedSalary
0,500,France,male,40,3,60000,2,1,1,50000


In [8]:
# Encode categorical variables
input_df['Gender'] = label_encoder_gender.fit_transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrcard,IsActiveMember,EstimatedSalary
0,500,France,0,40,3,60000,2,1,1,50000


In [9]:
df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrcard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,500,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
# Scaling the input data
input_scaled = scaler.fit_transform(df)
input_scaled

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [11]:
# Prediction
pred = model.predict(input_scaled)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step


array([[0.21117593]], dtype=float32)

In [20]:
pred_prob = pred[0][0]
pred_prob

np.float32(0.21117593)

In [23]:
if pred_prob > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")


The customer is unlikely to churn.
